In [1]:
#importar pyspark
import pyspark
from pyspark import SparkContext
sc =SparkContext()


In [2]:
#inicializar SQLContext
from pyspark import SQLContext
sqlContext = SQLContext(sc)

C:\Users\Default.DESKTOP-GVJJTQ5\anaconda3\lib\site-packages\pyspark\sql\context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [3]:
#importar datos
df = sqlContext.read.csv("heart.dat", sep = " ", inferSchema = True)

In [4]:
#revisar tipo de datos
df.printSchema()

root
 |-- _c0: double (nullable = true)
 |-- _c1: double (nullable = true)
 |-- _c2: double (nullable = true)
 |-- _c3: double (nullable = true)
 |-- _c4: double (nullable = true)
 |-- _c5: double (nullable = true)
 |-- _c6: double (nullable = true)
 |-- _c7: double (nullable = true)
 |-- _c8: double (nullable = true)
 |-- _c9: double (nullable = true)
 |-- _c10: double (nullable = true)
 |-- _c11: double (nullable = true)
 |-- _c12: double (nullable = true)
 |-- _c13: integer (nullable = true)



In [5]:
#Mostrar datos
df.show(5)

+----+---+---+-----+-----+---+---+-----+---+---+----+----+----+----+
| _c0|_c1|_c2|  _c3|  _c4|_c5|_c6|  _c7|_c8|_c9|_c10|_c11|_c12|_c13|
+----+---+---+-----+-----+---+---+-----+---+---+----+----+----+----+
|70.0|1.0|4.0|130.0|322.0|0.0|2.0|109.0|0.0|2.4| 2.0| 3.0| 3.0|   2|
|67.0|0.0|3.0|115.0|564.0|0.0|2.0|160.0|0.0|1.6| 2.0| 0.0| 7.0|   1|
|57.0|1.0|2.0|124.0|261.0|0.0|0.0|141.0|0.0|0.3| 1.0| 0.0| 7.0|   2|
|64.0|1.0|4.0|128.0|263.0|0.0|0.0|105.0|1.0|0.2| 2.0| 1.0| 7.0|   1|
|74.0|0.0|2.0|120.0|269.0|0.0|2.0|121.0|1.0|0.2| 1.0| 1.0| 3.0|   1|
+----+---+---+-----+-----+---+---+-----+---+---+----+----+----+----+
only showing top 5 rows



In [6]:
#Asignar nombres a variables
columns = ["anos", "sexo", "tipo_dolor", "presion_arterial_reposo", "colesterol_serico", "azucar", 
              "resl_electrocardiograficos", "freq_cardiaca_max", "angina", "oldpeak", "pendiente_st", "vasos_fluoroscopia",
             "thal", "target"]

for i in range(len(columns)):
    df = df.withColumnRenamed('_c'+str(i), columns[i])
    
df.columns

['anos',
 'sexo',
 'tipo_dolor',
 'presion_arterial_reposo',
 'colesterol_serico',
 'azucar',
 'resl_electrocardiograficos',
 'freq_cardiaca_max',
 'angina',
 'oldpeak',
 'pendiente_st',
 'vasos_fluoroscopia',
 'thal',
 'target']

In [7]:
df = df.drop('target')

In [8]:
#Agregar columna enfermo

# from pyspark.sql.functions import col
# x = df.select(col("thal"))

# #definir función
# def enfermo (x):
#     x.map(lambda x: 1 if x == "7.0" else 0)

# df.transform(enfermo, x)

from pyspark.sql.functions import  when
df = df.withColumn('enfermo', when(df.thal>6, 1).otherwise(0))
    

In [9]:
#revisar tipo de datos
df.printSchema()

root
 |-- anos: double (nullable = true)
 |-- sexo: double (nullable = true)
 |-- tipo_dolor: double (nullable = true)
 |-- presion_arterial_reposo: double (nullable = true)
 |-- colesterol_serico: double (nullable = true)
 |-- azucar: double (nullable = true)
 |-- resl_electrocardiograficos: double (nullable = true)
 |-- freq_cardiaca_max: double (nullable = true)
 |-- angina: double (nullable = true)
 |-- oldpeak: double (nullable = true)
 |-- pendiente_st: double (nullable = true)
 |-- vasos_fluoroscopia: double (nullable = true)
 |-- thal: double (nullable = true)
 |-- enfermo: integer (nullable = false)



In [10]:
#Convertir enfermo a string
#from pyspark.sql import types

# CONTI_FEATURES  = ["colesterol_serico", "freq_cardiaca_max", "oldpeak", "pendiente_st"]
# # # Convert the type
# for i in CONTI_FEATURES:
#     df = df.withColumn(i, df[i].cast(types.FloatType()))

# INT_FEATURES = ["anos", "vasos_fluoroscopia", "sexo ", ", "azucar""]
# for i in INT_FEATURES:
#     df = df.withColumn(i, df[i].cast(types.IntegerType()))

#df = df.withColumn("enfermo", df["enfermo"].cast(types.StringType()))

In [11]:
#revisar tipo de datos
df.printSchema()

root
 |-- anos: double (nullable = true)
 |-- sexo: double (nullable = true)
 |-- tipo_dolor: double (nullable = true)
 |-- presion_arterial_reposo: double (nullable = true)
 |-- colesterol_serico: double (nullable = true)
 |-- azucar: double (nullable = true)
 |-- resl_electrocardiograficos: double (nullable = true)
 |-- freq_cardiaca_max: double (nullable = true)
 |-- angina: double (nullable = true)
 |-- oldpeak: double (nullable = true)
 |-- pendiente_st: double (nullable = true)
 |-- vasos_fluoroscopia: double (nullable = true)
 |-- thal: double (nullable = true)
 |-- enfermo: integer (nullable = false)



In [12]:
#Crear variable feature
from pyspark.ml.feature import VectorAssembler

# Specify the input and output columns of the vector assembler
assembler = VectorAssembler(
    inputCols=['anos',
               'sexo',
               'tipo_dolor',
               'presion_arterial_reposo',
               'colesterol_serico',
               'azucar',
               'resl_electrocardiograficos',
               'freq_cardiaca_max',
               'angina',
               'oldpeak',
               'pendiente_st',
               'vasos_fluoroscopia'        
    ],
    outputCol='features'
)

# Transform the data
df = assembler.transform(df)


In [13]:
df.show(5)

+----+----+----------+-----------------------+-----------------+------+--------------------------+-----------------+------+-------+------------+------------------+----+-------+--------------------+
|anos|sexo|tipo_dolor|presion_arterial_reposo|colesterol_serico|azucar|resl_electrocardiograficos|freq_cardiaca_max|angina|oldpeak|pendiente_st|vasos_fluoroscopia|thal|enfermo|            features|
+----+----+----------+-----------------------+-----------------+------+--------------------------+-----------------+------+-------+------------+------------------+----+-------+--------------------+
|70.0| 1.0|       4.0|                  130.0|            322.0|   0.0|                       2.0|            109.0|   0.0|    2.4|         2.0|               3.0| 3.0|      0|[70.0,1.0,4.0,130...|
|67.0| 0.0|       3.0|                  115.0|            564.0|   0.0|                       2.0|            160.0|   0.0|    1.6|         2.0|               0.0| 7.0|      1|[67.0,0.0,3.0,115...|
|57.0| 1.0

In [14]:
from pyspark.ml.feature import StandardScaler

# estandarizar
scaler = StandardScaler(inputCol="features", outputCol="scaledFeatures", withStd=True, withMean=False)
scalerModel = scaler.fit(df)
df = scalerModel.transform(df)

In [15]:
df.show(5)

+----+----+----------+-----------------------+-----------------+------+--------------------------+-----------------+------+-------+------------+------------------+----+-------+--------------------+--------------------+
|anos|sexo|tipo_dolor|presion_arterial_reposo|colesterol_serico|azucar|resl_electrocardiograficos|freq_cardiaca_max|angina|oldpeak|pendiente_st|vasos_fluoroscopia|thal|enfermo|            features|      scaledFeatures|
+----+----+----------+-----------------------+-----------------+------+--------------------------+-----------------+------+-------+------------+------------------+----+-------+--------------------+--------------------+
|70.0| 1.0|       4.0|                  130.0|            322.0|   0.0|                       2.0|            109.0|   0.0|    2.4|         2.0|               3.0| 3.0|      0|[70.0,1.0,4.0,130...|[7.68465131046640...|
|67.0| 0.0|       3.0|                  115.0|            564.0|   0.0|                       2.0|            160.0|   0.0| 

In [16]:
# Select only the 'features' and 'new_column' for the model
model_data = df.select('features', 'enfermo')

# Rename 'new_column' to 'label' as required by MLlib
model_data = model_data.withColumnRenamed('enfermo', 'label')

# Split the data into training and test sets
train_data, test_data = model_data.randomSplit([0.5, 0.5])

In [17]:
train_data.show(5)

+--------------------+-----+
|            features|label|
+--------------------+-----+
|(12,[0,2,3,4,7,10...|    0|
|(12,[0,2,3,4,7,10...|    0|
|(12,[0,2,3,4,7,10...|    0|
|(12,[0,2,3,4,7,10...|    0|
|(12,[0,2,3,4,7,10...|    0|
+--------------------+-----+
only showing top 5 rows



In [18]:
from pyspark.ml.classification import LogisticRegression
# Create a Logistic Regression model and fit it to the training data
lr = LogisticRegression()
lr_model = lr.fit(train_data)

# Make predictions on the test data
predictions = lr_model.transform(test_data)

# Show some predictions
predictions.show()

+--------------------+-----+--------------------+--------------------+----------+
|            features|label|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|(12,[0,2,3,4,7,10...|    0|[3.14526709614831...|[0.95872182662437...|       0.0|
|(12,[0,2,3,4,7,10...|    0|[2.50233666267183...|[0.92430546645575...|       0.0|
|[34.0,1.0,1.0,118...|    0|[3.47057858151705...|[0.96983894741387...|       0.0|
|[35.0,0.0,4.0,138...|    0|[2.19263853535136...|[0.89958649841182...|       0.0|
|[35.0,1.0,4.0,120...|    1|[0.06724021769477...|[0.51680372375209...|       0.0|
|[40.0,1.0,1.0,140...|    1|[2.36273187005977...|[0.91394091817381...|       0.0|
|[40.0,1.0,4.0,110...|    1|[1.18160972373368...|[0.76523711292427...|       0.0|
|[40.0,1.0,4.0,152...|    1|[0.19117629141765...|[0.54764903660952...|       0.0|
|[41.0,1.0,2.0,110...|    0|[2.04802247115920...|[0.88574764787751...|       0.0|
|[41.0,1.0,3.0,1

In [21]:
from pyspark.sql import functions

In [22]:
predictions = predictions.withColumn('correct', functions.expr('label == prediction'))

In [23]:
predictions.show(5)

+--------------------+-----+--------------------+--------------------+----------+-------+
|            features|label|       rawPrediction|         probability|prediction|correct|
+--------------------+-----+--------------------+--------------------+----------+-------+
|(12,[0,2,3,4,7,10...|    0|[3.14526709614831...|[0.95872182662437...|       0.0|   true|
|(12,[0,2,3,4,7,10...|    0|[2.50233666267183...|[0.92430546645575...|       0.0|   true|
|[34.0,1.0,1.0,118...|    0|[3.47057858151705...|[0.96983894741387...|       0.0|   true|
|[35.0,0.0,4.0,138...|    0|[2.19263853535136...|[0.89958649841182...|       0.0|   true|
|[35.0,1.0,4.0,120...|    1|[0.06724021769477...|[0.51680372375209...|       0.0|  false|
+--------------------+-----+--------------------+--------------------+----------+-------+
only showing top 5 rows

